# Engenharia de Atributos

Importando as dependências

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Lendo os dados brutos e concatenando-os em um único dataframe

In [65]:
base_path = '../data/raw/' 

dados_2021 = pd.read_csv(base_path + '2021.csv', encoding='latin-1', sep=';')
dados_2022 = pd.read_csv(base_path + '2022.csv', sep=',')
dados_2023 = pd.read_csv(base_path + '2023.csv', encoding='latin-1', sep=';')
dados_2024 = pd.read_csv(base_path + '2024.csv', sep=',')

dados = pd.concat([dados_2021, dados_2022, dados_2023, dados_2024], axis=0, ignore_index=True)

dados.shape

(250889, 30)

Na análise exploratória, foram indentificadas colunas em que haviam dados nulos:

In [66]:
nulos = dados.isnull().sum().sort_values(ascending=False)
nulos[nulos>0]

uop                       219
delegacia                  85
regional                   23
classificacao_acidente      4
dtype: int64

Reconhecendo os dados dessas colunas

In [67]:
dados['uop'].head(5), dados['delegacia'].head(5), dados['regional'].head(5), dados['classificacao_acidente'].head(5)

(0    UOP01-DEL05-SP
 1    UOP01-DEL01-AM
 2    UOP03-DEL04-CE
 3    UOP02-DEL04-RJ
 4    UOP04-DEL05-RJ
 Name: uop, dtype: object,
 0    DEL05-SP
 1    DEL01-AM
 2    DEL04-CE
 3    DEL04-RJ
 4    DEL05-RJ
 Name: delegacia, dtype: object,
 0    SPRF-SP
 1    SPRF-AM
 2    SPRF-CE
 3    SPRF-RJ
 4    SPRF-RJ
 Name: regional, dtype: object,
 0    Com Vítimas Feridas
 1                    NaN
 2    Com Vítimas Feridas
 3    Com Vítimas Feridas
 4    Com Vítimas Feridas
 Name: classificacao_acidente, dtype: object)

Substituindo os dados faltantes pela moda das respectivas colunas

In [68]:
valores = {
    'uop': dados['uop'].mode()[0],
    'delegacia': dados['delegacia'].mode()[0],
    'regional': dados['regional'].mode()[0],
    'classificacao_acidente': dados['classificacao_acidente'].mode()[0]
}

valores

{'uop': 'UOP01-DEL01-SC',
 'delegacia': 'DEL01-PR',
 'regional': 'SPRF-MG',
 'classificacao_acidente': 'Com Vítimas Feridas'}

In [69]:
dados = dados.fillna(value=valores)

Checando se os valores nulos foram realmente preenchidos. Como nenhuma coluna com valor nulo é impressa, não existem mais valores nulos na base.

In [70]:
nulos = dados.isnull().sum().sort_values(ascending=False)
nulos[nulos>0]


Series([], dtype: int64)

Tratamento de data/hora, mudando para o formato datetime